## Setup

In [1]:
# Standard Imports
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import os


In [2]:
# Error settings
# Use NaN as actual data type instead of just error condition
np.seterr(divide='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

## Load

In [3]:
from datetime import date

df = pd.read_csv("~/projects/covid/data/timeseries.csv")
df["date"] = pd.to_datetime(df["date"])
df["date_no"] = (df["date"] - pd.Timestamp('2020-01-01 00:00:00')).dt.days
df["per_capita"] = df["cases"] / df["population"]

print(df.columns)

Index(['name', 'level', 'city', 'county', 'state', 'country', 'population',
       'lat', 'long', 'url', 'aggregate', 'tz', 'cases', 'deaths', 'recovered',
       'active', 'tested', 'hospitalized', 'discharged', 'growthFactor',
       'date', 'date_no', 'per_capita'],
      dtype='object')


/home/tianshu/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Analysis

In [4]:
from pymc3.ode import DifferentialEquation
import pymc3 as pm

In [5]:
def sir_ode(y, t, p, N):
    # y = [S, I, R]; p = [beta, gamma]
    return (
        - p[0] * y[0] * y[1] / N,
        p[0] * y[0] * y[1] / N - p[1] * y[1],
        p[1] * y[1]
    )

def sample(data, start=21, end=0):

    population = data["population"].iloc[0]
    R_true = np.array(data.sort_values("date_no")[-start:-end]["cases"])

    sir = DifferentialEquation(
        func=lambda y, t, p: sir_ode(y, t, p, population),
        times=np.arange(0, start - end, 1),
        n_states=3, n_theta=2, t0=0)

    with pm.Model() as model:
        gamma = pm.Gamma('gamma', 2, 2)
        beta = pm.Gamma('beta', 2, 2)
        i_0 = pm.Poisson('i_0', int(R_true[4] - R_true[0]))

        prior_start = [population - i_0 - R_true[0], i_0, R_true[0]]
        sir_mean = sir(y0=prior_start, theta=[beta, gamma])
        Y = pm.Poisson("Y", mu=sir_mean[:, 2], observed=R_true)

        trace = pm.sample(draws=500, tune=500, step=pm.NUTS(), chains=1)

    return trace

In [6]:
def sample_place(name):
    _df = df[df["name"] == name]
    trace = sample(_df, start=26, end=5)
    np.savez(
        "./results/trace_{}.npz".format(name),
        gamma=trace["gamma"], beta=trace["beta"], i_0=trace["i_0"])

In [9]:
places_qualified = df[
    (df["cases"] > 100) &
    (df["date_no"] < df["date_no"].max() - 26)
]["name"].unique()
countries = df[df["name"].isin(places_qualified) & (df["level"] == "country")]["name"].unique()

In [10]:
def sample_countries():

    for country in countries:
        print(country)
        if "trace_{}.npz".format(country) not in os.listdir("./results"):
            try:
                sample_place(country)
            except Exception as e:
                print("EXCEPTION:")
                print(e)

In [11]:
counties_qualified = df[
    (df["cases"] > 50) &
    (df["date_no"] < df["date_no"].max() - 26)
]["name"].unique()

top_states = df[
    df["name"].isin(counties_qualified) &
    (df["level"] == "county") &
    (df["country"] == 'United States') &
    (df["date_no"] == 114)
]["state"].value_counts()

In [12]:
print(top_states[:10])

New York         21
New Jersey       17
California       16
Pennsylvania     12
Florida          12
Colorado         11
Massachusetts    10
Washington       10
Texas             9
Georgia           9
Name: state, dtype: int64


In [13]:
def sample_state(state):
    df_ = df[
        (df["level"] == "county") &
        (df["state"] == state) &
        (df["name"].isin(counties_qualified))
    ]
    
    targets = df_["name"].unique()
    for idx, place in enumerate(targets):
        print("[{}/{}] {}".format(idx + 1, len(targets), place))
        if("trace_{}.npz".format(place) not in os.listdir("./results")):
            sample_place(place)

In [67]:
sample_state("New York")

[1/21] Albany County, New York, United States


Sequential sampling (1 chains in 1 job)
CompoundStep
>NUTS: [beta, gamma]
>Metropolis: [i_0]
Sampling chain 0, 0 divergences:   0%|          | 0/1000 [00:00<?, ?it/s]/home/tianshu/.local/lib/python3.6/site-packages/scipy/integrate/odepack.py:248: ODEintWarning: Illegal input detected (internal error). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
Sampling chain 0, 0 divergences:  20%|█▉        | 196/1000 [00:44<03:18,  4.05it/s]/home/tianshu/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
Sampling chain 0, 0 divergences:  20%|█▉        | 196/1000 [00:44<03:02,  4.41it/s]
Only one chain was sampled, this makes it impossible to run some convergence checks


[2/21] Bronx County, New York, United States


Sequential sampling (1 chains in 1 job)
CompoundStep
>NUTS: [beta, gamma]
>Metropolis: [i_0]
Sampling chain 0, 0 divergences:   0%|          | 0/1000 [00:00<?, ?it/s]/home/tianshu/.local/lib/python3.6/site-packages/scipy/integrate/odepack.py:248: ODEintWarning: Illegal input detected (internal error). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
Sampling chain 0, 0 divergences:   2%|▏         | 18/1000 [00:02<03:11,  5.13it/s]/home/tianshu/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
Sampling chain 0, 0 divergences:   2%|▏         | 18/1000 [00:02<02:03,  7.95it/s]
Only one chain was sampled, this makes it impossible to run some convergence checks


[3/21] Dutchess County, New York, United States


Sequential sampling (1 chains in 1 job)
CompoundStep
>NUTS: [beta, gamma]
>Metropolis: [i_0]
Sampling chain 0, 0 divergences:   0%|          | 2/1000 [00:01<08:25,  1.98it/s]
Only one chain was sampled, this makes it impossible to run some convergence checks


[4/21] Erie County, New York, United States


KeyboardInterrupt: 

In [ ]:
sample_state("New Jersey")

In [ ]:
sample_state("California")

In [16]:
from tqdm import tqdm

for file in tqdm(os.listdir("./results")):
    fig, axs = plt.subplots(3, 2, figsize=(16, 12))

    trace = np.load(os.path.join("./results", file))
    for ax, var in zip(axs, ["i_0", "beta", "gamma"]):
        ax[0].plot(trace[var])
        ax[0].set_title(var)
        ax[1].hist(trace[var], bins=25)
        ax[1].set_title("mean={:.4f}".format(np.mean(trace[var])))

    fig.savefig('./traceplot/' + file.split(".")[0] + ".png")
    plt.close(fig)


100%|██████████| 104/104 [00:41<00:00,  2.50it/s]


In [40]:
import seaborn as sns

for file in tqdm(os.listdir("./results")):
    trace = np.load(os.path.join("./results", file))
    
    g = sns.PairGrid(pd.DataFrame({k: trace[k] for k in ['i_0', 'beta', 'gamma']}))
    g.map_diag(sns.kdeplot)
    g.map_offdiag(sns.kdeplot, shade=True)
    g.fig.suptitle(file)
    g.fig.savefig('./kdeplot/' + file.split(".")[0] + ".png")
    plt.close(g.fig)

100%|██████████| 104/104 [02:28<00:00,  1.43s/it]
